# resnet fuller

In [1]:
import keras
import csv
import numpy as np # linear algebra
import collections
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf

from PIL import Image 
from tqdm import tqdm
from numpy import asarray
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Activation, Dropout, Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization

Using TensorFlow backend.


## Load data, oversampling, train_test_split

In [2]:
import pandas as pd
import numpy as np
import cv2
data_path = 'fuller.csv'
image_size=(48, 48)

# Helper functions
def load_data(data_path):
    data_set = pd.read_csv(data_path)
    X = data_set[' pixels']
    y = data_set['emotion']
    return X, y

def oversampling(X, y):
    X = X.values.reshape(-1, 1)
    print('before oversampling:', collections.Counter(y))
    
    oversample = RandomOverSampler(sampling_strategy='auto')
    X_over, y_over = oversample.fit_resample(X.reshape(-1, 1), y)
    a = np.array(y_over)
    print('after oversampling:', collections.Counter(a))
    
    y_over = pd.Series(y_over)
    y_over= y_over.values.reshape(len(y_over),1)

    return X_over, y_over

def preprocessing(pixels):
    a = []
    
    for i in range(len(pixels)):
            image_string = (pixels)[i].split(' ') 
            image_data = np.asarray(image_string, dtype=np.uint8).reshape(48,48,1)
            a.append(image_data)

    return a

def reshape(X):
    print('before:', X.shape)
    a= []
    X = pd.Series(X.flatten())
    a = preprocessing(X)
    X = np.array(a)
    print('after:', X.shape)
    return X

In [3]:
## Start here ##
#Load data
X, y = load_data(data_path)

#Oversampling so each class has same number of examples
X_over, y_over = oversampling(X, y)

#train test split
X_train,X_test,y_train,y_test = train_test_split(X_over,y_over, test_size=0.2)

#Reshape to fit in as input to model
X_train = reshape(X_train)

before oversampling: Counter({3: 8989, 4: 6077, 2: 5121, 0: 4953, 5: 4002, 1: 547})
after oversampling: Counter({0: 8989, 2: 8989, 4: 8989, 3: 8989, 5: 8989, 1: 8989})
before: (43147, 1)
after: (43147, 48, 48, 1)


In [4]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = "4" #please put your GPU

In [5]:
model = models.Sequential()
model.add(layers.Conv2D(64, (1, 1), padding='same', activation='relu', input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(layers.Conv2D(128, (3, 3),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(layers.Conv2D(256, (5, 5),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(layers.MaxPooling2D((2, 2),padding="same"))
model.add(Dropout(0.25))


model.add(layers.Flatten())

model.add(layers.Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(layers.Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(layers.Dense(6, activation='softmax'))
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        128       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
dropout (Dropout)            (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 128)       512       
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 48, 128)       0         
_________________________________________

In [6]:
print(X_train.shape, y_train.shape)

(43147, 48, 48, 1) (43147, 1)


In [ ]:
# save the model
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('resnet_es.h5', save_best_only=True, monitor='val_loss', mode='min')

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=64, epochs=20, validation_split = 0.25, callbacks=[earlyStopping, mcp_save])

model.save('resnet_es')

Train on 32360 samples, validate on 10787 samples
Epoch 1/20
32360/32360 [==============================] - 3483s 108ms/sample - loss: 1.6105 - acc: 0.4163 - val_loss: 1.5152 - val_acc: 0.5206
Epoch 2/20
32360/32360 [==============================] - 3403s 105ms/sample - loss: 1.4699 - acc: 0.5704 - val_loss: 1.4417 - val_acc: 0.5977
Epoch 3/20
32360/32360 [==============================] - 3437s 106ms/sample - loss: 1.3951 - acc: 0.6468 - val_loss: 1.3961 - val_acc: 0.6441
Epoch 4/20
32360/32360 [==============================] - 3494s 108ms/sample - loss: 1.3428 - acc: 0.7005 - val_loss: 1.3706 - val_acc: 0.6699
Epoch 5/20
32360/32360 [==============================] - 3445s 106ms/sample - loss: 1.3050 - acc: 0.7384 - val_loss: 1.3562 - val_acc: 0.6817
Epoch 6/20
32360/32360 [==============================] - 3339s 103ms/sample - loss: 1.2715 - acc: 0.7716 - val_loss: 1.3330 - val_acc: 0.7063
Epoch 7/20
32360/32360 [==============================] - 3430s 106ms/sample - loss: 1.2425 

### Saving the model

In [ ]:
# save your model and weight (only submit best model)
best_model = model

model_json = best_model.to_json()
with open("resnet_fuller.json", "w") as json_file:
    json_file.write(model_json)
best_model.save_weights('resnet_fuller.h5')

### Testing

In [11]:
# Your model will be tested as following
import tensorflow as tf

# Model reconstruction from JSON file
with open('resnet_fuller.json', 'r') as json_file:
    json_savedModel= json_file.read()

test_model = tf.keras.models.model_from_json(json_savedModel)
test_model.summary()

test_model.compile(loss='sparse_categorical_crossentropy',
                   optimizer='adam',
                    metrics=['acc'])

# Load weights into the new model
test_model.load_weights('resnet_fuller.h5')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        128       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
dropout (Dropout)            (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 128)       512       
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 48, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 256)       8

In [13]:
import pandas as pd
import numpy as np
import cv2
data_path = 'test.csv'
image_size=(48, 48)

def load_data(data_path):
        data = pd.read_csv(data_path)
        pixels = data['pixels'].tolist()
        width, height = 48, 48
        faces = []
        for pixel_sequence in pixels:
            face = [int(pixel) for pixel in pixel_sequence.split(' ')]
            face = np.asarray(face).reshape(width, height)
            face = cv2.resize(face.astype('uint8'),image_size)
            faces.append(face.astype('float32'))
        faces = np.asarray(faces)
        faces = np.expand_dims(faces, -1)

        emotions = data.emotion.values.reshape(-1, 1)
        return faces, emotions
    
faces_test, emotions_test = load_data(data_path); 

In [14]:
## Testing
test_loss, test_acc = test_model.evaluate(faces_test, emotions_test) 
print('Test accuracy:', test_acc)

4794/4794 [==============================] - 29s 6ms/sample - loss: 2.2532 - acc: 0.8327
Test accuracy: 0.8327075


In [16]:
## Testing
X_test = reshape(X_test)
test_loss, test_acc = test_model.evaluate(X_test, y_test) 
print('Test accuracy:', test_acc)

before: (10787, 1)
after: (10787, 48, 48, 1)
10787/10787 [==============================] - 64s 6ms/sample - loss: 2.9323 - acc: 0.7648
Test accuracy: 0.7648095


In [1]:
import tensorflow as tf; print(tf.__version__)

1.15.0
